In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from keras import models
from keras.models import load_model
from utilities.gtsrb_loader.get_folderpath import get_folderpath
from utilities.gtsrb_loader.load_data import load_data

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
model = models.load_model('/data_on_server/Nets/trained/localization/2017-07-12_localization.h5')

In [ ]:
#path = get_folderpath(original_images=False, subset='test')
path="/data_on_server/Images/GTSRB_64x64_PiP/Final_Training"

In [ ]:
X_train, y_train, box = load_data(path)

In [ ]:
pics=np.array(X_train[1:1000], dtype=np.float32)/255
bboxes_list = np.array(box[1:1000], dtype=int)

In [ ]:
predictions = model.predict(np.array(pics), verbose=1)*64

In [ ]:
x, y = bboxes_list[:,0], bboxes_list[:,1]
width = bboxes_list[:,2] - bboxes_list[:,0]
heigth = bboxes_list[:,3] - bboxes_list[:,1] 

In [ ]:
x_pred, y_pred = predictions[:,0], predictions[:,1]
width_pred = predictions[:,2] - predictions[:,0]
heigth_pred = predictions[:,3] - predictions[:,1]

In [ ]:
for i, image in enumerate(pics[1:30]):
    ax = plt.subplot(5, 6, i +1)
    plt.imshow(image)
    plt.axis('off')
    # correct bounding boxes in yellow
    rect = patches.Rectangle((x[i], y[i]), width[i], heigth[i], linewidth=1, edgecolor='y', facecolor='none')
    ax.add_patch(rect)
    # predicted bounding boxes in red
    rect = patches.Rectangle((x_pred[i], y_pred[i]), width_pred[i], heigth_pred[i], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
plt.show()

In [ ]:
error = np.linalg.norm(predictions - bboxes_list, axis=1).reshape((30, 1))
pd.DataFrame(np.concatenate((predictions, bboxes_list, error), axis=1),
             columns=['x1', 'y1', 'x2', 'y2', 'x1 pred', 'y1 pred', 'x2 pred', 'y2 pred', 'l2 error'])

In [ ]:
image1 = plt.imread(os.path.join('utilities/image_split', '4713small.jpg'), format='jpeg')
image2 = plt.imread(os.path.join('utilities/image_split', '1057small.jpg'), format='jpeg')
pic_list2=[image1, image2]
predictions2 = np.rint(model.predict(np.array(pic_list2, dtype=np.float32)/255, verbose=1)*64).astype(int)
x_pred2, y_pred2 = predictions2[:,0], predictions2[:,1]
width_pred2 = predictions2[:,2] - predictions2[:,0]
heigth_pred2 = predictions2[:,3] - predictions2[:,1]

In [ ]:
for i, image in enumerate(pic_list2):
    ax = plt.subplot(5, 6, i +1)
    plt.imshow(image)
    plt.axis('off')
    # correct bounding boxes in yellow
    # predicted bounding boxes in red
    rect = patches.Rectangle((x_pred2[i], y_pred2[i]), width_pred2[i], heigth_pred2[i], linewidth=1, edgecolor='r', facecolor='none')
    ax.add_patch(rect)
plt.show()

In [ ]:
pd.DataFrame(predictions2,columns=['x1 pred', 'y1 pred', 'x2 pred', 'y2 pred'])

In [ ]:
predictions2
